In [146]:
import requests, json
import pandas as pd
from useful_funcs import *
from predict_player_scores import get_latest_players_df

In [85]:
league_url = 'https://draft.premierleague.com/api/league/19188/element-status'
r = requests.get(league_url).json()

In [92]:
ownership_df = pd.json_normalize(r['element_status'])

In [78]:
url = 'https://draft.premierleague.com/api/bootstrap-static'
req = requests.get(url).json()
players_df = pd.json_normalize(req['elements'])

In [ ]:
players_df['full_name'] = players_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)
merge_name_df = players_df[['id', 'full_name']]

In [96]:
ownership_df_name = ownership_df.merge(merge_name_df, how='left', left_on='element', right_on='id') 

In [100]:
old_pred_df = pd.read_csv('../drafting/player_score_predictions_delta.csv')

In [107]:
old_pred_df['full_name'] = old_pred_df['full_name'].str.lower().str.strip()

In [138]:
pred_df = old_pred_df.merge(ownership_df_name, how='outer', on='full_name')

In [139]:
pred_df

,full_name,pos,team_name,total_points,y_pred,notes,y_pred_delta_1,y_pred_delta_2,y_pred_delta_3,y_pred_delta_4,...,y_pred_delta_7,y_pred_delta_8,y_pred_delta_9,y_pred_delta_10,is_available,element,in_accepted_trade,owner,status,id
0,aaron_anselmino,DEF,Chelsea,0.0,-1.800599,NaN,0.625922,0.0,0.000000,0.000000,...,0.000000,8.871476,0.000000,0.000000,True,234,False,NaN,a,234
1,aaron_hickey,DEF,Brentford,0.0,5.340009,NaN,0.000000,0.0,0.000000,0.611548,...,0.000000,0.000000,0.000000,0.000000,True,116,False,NaN,a,116
2,aaron_ramsdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,673,False,NaN,a,673
3,aaron_ramsey,MID,Burnley,0.0,27.162412,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,True,211,False,NaN,a,211
4,aaron_wan_bissaka,DEF,West Ham,118.0,56.569787,NaN,0.000000,0.0,0.000000,0.000000,...,1.872975,0.000000,0.000000,0.000000,True,610,False,85076.0,o,610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,zain_silcott_duberry,MID,Bournemouth,1.0,24.922663,jan_transfer,0.086670,0.0,0.000000,0.000000,...,0.000000,0.000000,0.421542,0.000000,True,95,False,NaN,a,95
681,zak_johnson,DEF,Sunderland,0.0,33.661237,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.625922,True,538,False,NaN,a,538
682,zeki_amdouni,FWD,Burnley,0.0,66.568110,promoted_team,0.000000,0.0,0.000000,0.970461,...,5.550651,1.220135,0.000000,0.356870,True,216,False,NaN,a,216
683,zepiqueno_redmond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,65,False,NaN,a,65


In [206]:
pos_pred_df2 = pred_df.sort_values('y_pred', ascending=False).query('status=="a"')

pos_pred_df3 = pos_pred_df2.iloc[:, :5]

In [207]:
updated_player_df = get_latest_players_df()

col_of_interest = ['first_name', 'second_name',
                   'form', 'total_points', 'transfers_in', 'transfers_out', 'value_form', 'minutes']

current_stats = updated_player_df[col_of_interest]

current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)

current_stats2 = current_stats.drop(['first_name', 'second_name'], axis=1)

current_stats2['transfer_delta'] = current_stats2['transfers_in'] - current_stats2['transfers_out']

C:\Users\User\AppData\Local\Temp\ipykernel_9528\1447550088.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)


In [208]:
pos_pred_df4 = pos_pred_df3.merge(current_stats2, on='full_name', how='left')

In [209]:
pos_pred_df4


,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
0,son_heung_min,MID,Spurs,129.0,165.209198,0.0,0,0,14876,0.0,0,-14876
1,luis_diaz_marulanda,MID,Liverpool,183.0,144.874270,0.0,0,0,3908,0.0,0,-3908
2,marcus_rashford,MID,Man Utd,95.0,117.646442,0.0,0,0,599,0.0,0,-599
3,justin_kluivert,MID,Bournemouth,158.0,115.192051,0.0,0,258,29927,0.0,0,-29669
4,savio_moreira_de_oliveira,MID,Man City,106.0,111.910340,0.0,0,208,37903,0.0,0,-37695
...,...,...,...,...,...,...,...,...,...,...,...,...
590,will_lankshear,NaN,NaN,NaN,NaN,0.0,0,0,1700,0.0,0,-1700
591,yang_min_hyeok,NaN,NaN,NaN,NaN,0.0,0,0,1003,0.0,0,-1003
592,yunus_emre_konak,NaN,NaN,NaN,NaN,0.0,0,410,1544,0.0,0,-1134
593,zach_marsh,NaN,NaN,NaN,NaN,0.0,0,1745,2817,0.0,0,-1072


In [218]:
pos_pred_df4.sort_values('transfers_in', ascending=False).query('pos=="MID"').head(50)

,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
148,tijjani_reijnders,MID,Man City,0.0,57.748707,10.0,10,888183,28314,1.8,90,859869
160,matt_o'riley,MID,Brighton,50.0,57.205041,10.0,10,76922,13474,1.8,87,63448
13,declan_rice,MID,Arsenal,127.0,95.997649,6.0,6,61293,49586,0.9,82,11707
441,hamed_traore,MID,Bournemouth,0.0,24.922663,4.0,4,54507,12538,0.9,16,41969
121,elliot_anderson,MID,Nott'm Forest,107.0,61.030419,6.0,6,40836,47551,1.1,90,-6715
133,moises_caicedo_corozo,MID,Chelsea,98.0,59.919443,5.0,5,35183,48817,0.9,90,-13634
111,simon_adingra,MID,Sunderland,60.0,63.270167,6.0,6,32135,13140,1.1,75,18995
346,pape_matar_sarr,MID,Spurs,71.0,38.832055,6.0,6,29899,26329,1.2,90,3570
293,amadou_onana,MID,Aston Villa,65.0,42.287107,5.0,5,29414,17561,1.0,90,11853
61,dan_ndoye,MID,Nott'm Forest,0.0,79.084296,9.0,9,28162,7774,1.5,78,20388
